<a href="https://colab.research.google.com/github/LuisFTacla/WDW_Attendance_Calculator/blob/main/Tratamento_de_dados_Thrill_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
import scipy.interpolate as interpolate

In [2]:
# FUNÇÃO DE CONVERSÃO STR PARA DATETIME
def to_datetime(data):
  data['date_time'] = pd.to_datetime(data['Date/Time'])
  data['wait_time'] = data['Wait Time']
  data.drop(columns = ['Date/Time', 'Wait Time', 'Ride'], inplace = True)
  return data


# DEFININDO LIMITES PARA OS DADOS
date_begin = pd.to_datetime('2019-3-1')
date_end = pd.to_datetime('2020-3-1')


# LEITURA DOS ARQUIVOS
Astro_Orbiter_df = {1: pd.read_csv('https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Dados/ThrillData/Magic_Kingdom/Astro_Orbiter/astroorbiter_01.csv'),
                    2: pd.read_csv('https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Dados/ThrillData/Magic_Kingdom/Astro_Orbiter/astroorbiter_02.csv'),
                    3: pd.read_csv('https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Dados/ThrillData/Magic_Kingdom/Astro_Orbiter/astroorbiter_03.csv'),
                    4: pd.read_csv('https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Dados/ThrillData/Magic_Kingdom/Astro_Orbiter/astroorbiter_04.csv'),
                    5: pd.read_csv('https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Dados/ThrillData/Magic_Kingdom/Astro_Orbiter/astroorbiter_05.csv'),
                    6: pd.read_csv('https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Dados/ThrillData/Magic_Kingdom/Astro_Orbiter/astroorbiter_06.csv'),
                    7: pd.read_csv('https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Dados/ThrillData/Magic_Kingdom/Astro_Orbiter/astroorbiter_07.csv'),
                    8: pd.read_csv('https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Dados/ThrillData/Magic_Kingdom/Astro_Orbiter/astroorbiter_08.csv'),
                    9: pd.read_csv('https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Dados/ThrillData/Magic_Kingdom/Astro_Orbiter/astroorbiter_09.csv'),
                    10: pd.read_csv('https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Dados/ThrillData/Magic_Kingdom/Astro_Orbiter/astroorbiter_10.csv'),
                    11: pd.read_csv('https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Dados/ThrillData/Magic_Kingdom/Astro_Orbiter/astroorbiter_11.csv'),
                    12: pd.read_csv('https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Dados/ThrillData/Magic_Kingdom/Astro_Orbiter/astroorbiter_12.csv')}


# UNIÃO DOS ARQUIVOS
Astro_Orbiter_df = Astro_Orbiter_df[1].append(Astro_Orbiter_df[2]).append(Astro_Orbiter_df[3]).append(Astro_Orbiter_df[4]).append(Astro_Orbiter_df[5]).append(Astro_Orbiter_df[6]).append(Astro_Orbiter_df[7]).append(Astro_Orbiter_df[8]).append(Astro_Orbiter_df[9]).append(Astro_Orbiter_df[10]).append(Astro_Orbiter_df[11]).append(Astro_Orbiter_df[12])


# CONVERSÃO DAS STR EM DATETIME
to_datetime(Astro_Orbiter_df)


# ORDENANDO DADOS PELA DATA
Astro_Orbiter_df.sort_values(by=['date_time'], inplace = True)


# CRIA COLUNA 'DELETE'
Astro_Orbiter_df['delete'] = 0


# RESETA ÍNDICE
Astro_Orbiter_df.reset_index(drop = True, inplace = True)


# SELECIONA LINHAS PARA APAGAR
for i in range(0, len(Astro_Orbiter_df)):
  if Astro_Orbiter_df['date_time'][i] < date_begin:
    Astro_Orbiter_df['delete'][i] = 1
  elif Astro_Orbiter_df['date_time'][i] > date_end:
    Astro_Orbiter_df['delete'][i] = 1


# APLICANDO FILTRO PARA APAGAR LINHAS
filtro = Astro_Orbiter_df.delete == 0
Astro_Orbiter_df = Astro_Orbiter_df[filtro]


# RESETANDO ÍNDICE NOVAMENTE
Astro_Orbiter_df.reset_index(drop = True, inplace = True)


# CRIA DATAFRAME BASE
five_min = pd.date_range(start='3/1/2019', end = '3/1/2020', freq = '5min')
base_df = pd.DataFrame(index = range(0,105409), columns = ['date_time', 'wait_time', 'delete'])
base_df['date_time'] = five_min
base_df['wait_time'] = 0
base_df['delete'] = 0


# UNINDO OS DADOS AO BASE
Astro_Orbiter_df = base_df.append(Astro_Orbiter_df)


# ORDENANDO DADOS PELA DATA
Astro_Orbiter_df.sort_values(by=['date_time'], inplace = True)


# RESETA ÍNDICE
Astro_Orbiter_df.reset_index(drop = True, inplace = True)


# SELECIONANDO LINHAS PARA REMOVER
for i in range(0,len(Astro_Orbiter_df)):
    if i != 0 and i != 1 and i != (len(Astro_Orbiter_df) - 1) and i != (len(Astro_Orbiter_df) - 2):
      if (Astro_Orbiter_df.wait_time[i] == 0):
        c1 = (Astro_Orbiter_df.wait_time[i + 1] != 0)
        c2 = (Astro_Orbiter_df.wait_time[i - 1] != 0)
        c3 = (Astro_Orbiter_df.wait_time[i + 2] != 0)
        c4 = (Astro_Orbiter_df.wait_time[i - 2] != 0)
        if c1 or c2 or c3 or c4:
          Astro_Orbiter_df.delete[i] = 1


# FILTRANDO DADOS
filtro2 = Astro_Orbiter_df.delete == 0
Astro_Orbiter_df = Astro_Orbiter_df[filtro2]


# INTERPOLANDO DADOS
Astro_Orbiter = base_df

X_interp = base_df.date_time
X = Astro_Orbiter_df.date_time
Y = Astro_Orbiter_df.wait_time
  
Y_interp = np.interp(X_interp, X, Y)
Astro_Orbiter.wait_time = np.around(Y_interp, decimals = 0)


# APAGANDO COLUNA 'DELETE'
Astro_Orbiter.drop(columns = ['delete'], inplace = True)


# CRIANDO COLUNA 'STATUS'
Astro_Orbiter['status'] = 0
for i in range(0, len(Astro_Orbiter)):
  if Astro_Orbiter.wait_time[i] == 0:
    Astro_Orbiter.status[i] = 0
  else:
    Astro_Orbiter.status[i] = 1









/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:122: SettingWithCopyWarning: 
A value is trying to 

In [3]:
Astro_Orbiter.shape

(105409, 3)

In [4]:
Astro_Orbiter.head(50)

,date_time,wait_time,status
0,2019-03-01 00:00:00,0.0,0
1,2019-03-01 00:05:00,0.0,0
2,2019-03-01 00:10:00,0.0,0
3,2019-03-01 00:15:00,0.0,0
4,2019-03-01 00:20:00,0.0,0
5,2019-03-01 00:25:00,0.0,0
6,2019-03-01 00:30:00,0.0,0
7,2019-03-01 00:35:00,0.0,0
8,2019-03-01 00:40:00,0.0,0
9,2019-03-01 00:45:00,0.0,0


In [5]:
Astro_Orbiter.tail(50)

,date_time,wait_time,status
105359,2020-02-29 19:55:00,50.0,1
105360,2020-02-29 20:00:00,60.0,1
105361,2020-02-29 20:05:00,60.0,1
105362,2020-02-29 20:10:00,60.0,1
105363,2020-02-29 20:15:00,26.0,1
105364,2020-02-29 20:20:00,20.0,1
105365,2020-02-29 20:25:00,35.0,1
105366,2020-02-29 20:30:00,35.0,1
105367,2020-02-29 20:35:00,50.0,1
105368,2020-02-29 20:40:00,50.0,1
